In [1]:
import warnings
warnings.filterwarnings('ignore')
# import cv2
import numpy as np
import glob
from keras.models import load_model
import tensorflow as tf
from keras import optimizers
import keras
import segmentation_models as sm

from architecture import create_model, create_callbacks


BACKUP_MODEL_PATH = "./../Models/backup.h5"
seed = 56

IMAGE_HEIGHT = IMAGE_WIDTH = 256 * 2 # * 2

EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 4

ROOT = "./../Data/Hannover/"
TRAIN_DIR = "_Train512/"

Segmentation Models: using `keras` framework.


In [2]:
model = create_model(IMAGE_HEIGHT, IMAGE_WIDTH)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='./../Models/unet_struct.png',show_shapes=True, show_layer_names=True)

from IPython.display import Image as IP_Image
IP_Image(retina=True, filename='./../Models/unet_struct.png')

In [2]:
opt = keras.optimizers.Adam(LEARNING_RATE)
callbacks = create_callbacks(BACKUP_MODEL_PATH)

model.compile(
      optimizer=opt,
      loss=sm.losses.DiceLoss(),
      metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore()])

NameError: name 'model' is not defined

In [3]:
import get_data_generators

augmentation = {
"featurewise_center": "False",
"samplewise_center": "False",
"featurewise_std_normalization": "False",
"samplewise_std_normalization": "False",
# "zca_whitening": "False",
# "zca_epsilon": "False",
"rotation_range": 0,
# "width_shift_range": "False",
# "height_shift_range": "False",
# "shear_range": "False",
# "zoom_range": "False",
# "channel_shift_range": "False",
# "fill_mode"],
# "cval": "False",
"horizontal_flip": "False",
"vertical_flip": "False",
#"rescale"]),
"validation_split": 0.08
}

train_gen, val_gen = get_data_generators.getDataGenerators(augmentation, (IMAGE_WIDTH, IMAGE_HEIGHT), ROOT + TRAIN_DIR + "Images", ROOT + TRAIN_DIR + "Masks", batch_size=BATCH_SIZE, seed=seed)

Found 3607 images belonging to 1 classes.
Found 313 images belonging to 1 classes.
Found 3607 images belonging to 1 classes.
Found 313 images belonging to 1 classes.


In [4]:
"""Load Model backup to resume training after crash or break or for pretraining"""
model = load_model(BACKUP_MODEL_PATH, custom_objects={"dice_loss" : sm.losses.DiceLoss(), "iou_score": sm.metrics.IOUScore(threshold=0.5), "f1-score":sm.metrics.FScore()})

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

history = model.fit(train_gen.data,  validation_data=val_gen.data, steps_per_epoch=train_gen.samplesPerBatch, validation_steps=val_gen.samplesPerBatch, callbacks=list(callbacks), epochs=EPOCHS, verbose=1) #,

Num GPUs Available:  1
Epoch 1/100
902/901 [==============================] - ETA: 0s - loss: 0.8443 - iou_score: 0.0858 - f1-score: 0.1557
Epoch 1: val_loss improved from inf to 0.82045, saving model to ./../Models\backup.h5
901/901 [==============================] - 118s 124ms/step - loss: 0.8443 - iou_score: 0.0858 - f1-score: 0.1557 - val_loss: 0.8205 - val_iou_score: 0.0997 - val_f1-score: 0.1791 - lr: 1.0000e-06
Epoch 2/100
902/901 [==============================] - ETA: 0s - loss: 0.8263 - iou_score: 0.0965 - f1-score: 0.1738
Epoch 2: val_loss improved from 0.82045 to 0.81109, saving model to ./../Models\backup.h5
901/901 [==============================] - 108s 120ms/step - loss: 0.8263 - iou_score: 0.0965 - f1-score: 0.1738 - val_loss: 0.8111 - val_iou_score: 0.1046 - val_f1-score: 0.1875 - lr: 1.0000e-06
Epoch 3/100
902/901 [==============================] - ETA: 0s - loss: 0.8148 - iou_score: 0.1034 - f1-score: 0.1852
Epoch 3: val_loss improved from 0.81109 to 0.79942, saving

In [7]:
import datetime

print("Time of saving model:", datetime.datetime.now())

model_path = "./../Models/Binary/512/bike_mapper_pre-train-plain-lr1e-4_Train3607_Val313_Test980_No-Aug.h5"
model.save(model_path)

Time of saving model: 2022-10-23 19:59:31.806682


In [4]:
from matplotlib import pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig(model + ".png")

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()
plt.savefig(model + ".png")

AttributeError: 'NoneType' object has no attribute 'history'

In [4]:
#Source: https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2
from keras import backend as K
# def iou_coef(y_true, y_pred, smooth=1):
#   intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
#   union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
#   iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
#
#   return iou
#
# def dice_coef(y_true, y_pred, smooth = 1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
#
# def soft_dice_loss(y_true, y_pred):
#     return 1-dice_coef(y_true, y_pred)